In [ ]:
import os
from pathlib import Path

from syft_rds.orchestra import setup_rds_server

import syft_flwr

## Step 1: Setup RDS Server

In [ ]:
ds_stack = setup_rds_server(email="ds@openmined.org", key="flwr")

In [ ]:
DO1 = "do1@openmined.org"
DO2 = "do2@openmined.org"
DS = "ds@openmined.org"

In [ ]:
do_client_1 = ds_stack.init_session(host=DO1)
assert do_client_1.host == DO1
assert do_client_1.email == DS
assert not do_client_1.is_admin

do_client_2 = ds_stack.init_session(host=DO2)
assert do_client_2.host == DO2
assert do_client_2.email == DS
assert not do_client_2.is_admin

datasites = [do_client_1, do_client_2]

## Step 2: Explore Datasets

In [ ]:
EXAMPLE = "quickstart-pytorch"
SYFTBOX_DATASET_NAME = "cifar10"

PROJECT_DIR = Path(os.getcwd()).parent.parent.absolute()
FLWR_PROJECT_PATH = PROJECT_DIR / "examples" / EXAMPLE

In [ ]:
dataset1 = do_client_1.dataset.get(name=SYFTBOX_DATASET_NAME)
dataset1.describe()

In [ ]:
dataset2 = do_client_2.dataset.get(name=SYFTBOX_DATASET_NAME)
dataset2.describe()

## Step 3: Bootstrap Project

In [ ]:
try:
    syft_flwr.bootstrap(FLWR_PROJECT_PATH, aggregator=DS, datasites=[DO1, DO2])
    print("Bootstrapped project successfully ✅")
except Exception as e:
    print(e)

## Step 4: DS Submits Jobs

In [ ]:
# Job submission

for client in datasites:
    job = client.jobs.submit(
        name="Syft Flower Pytorch Experiment",
        description="Syft Flower Federated Learning Experiment",
        user_code_path=FLWR_PROJECT_PATH,
        dataset_name=SYFTBOX_DATASET_NAME,
        tags=["federated", "learning", "syft", "flwr"],
        entrypoint="main.py",
    )
    print(job)

Now DOs can review the jobs

## Run simulation using the `main.py` file directly

In [ ]:
os.environ["SYFTBOX_CLIENT_CONFIG_PATH"] = str(ds_stack.client.config_path)
!uv run {str(FLWR_PROJECT_PATH / "main.py")} --active